In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/MyDrive

/content/drive/MyDrive


In [3]:
!pip install transformers
!pip install sentencepiece
!pip install bertopic

In [4]:
!pip install bertopic[visualization]

In [5]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-7lzdw4ms/kobert-tokenizer_9bb1193f64bb4866b10aeaebf707553b
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-7lzdw4ms/kobert-tokenizer_9bb1193f64bb4866b10aeaebf707553b
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [6]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [7]:
import torch
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
out = model(input_ids = torch.tensor(inputs['input_ids']), attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape

torch.Size([1, 768])

In [8]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [9]:
inputs = tokenizer.batch_encode_plus([text], return_tensors='pt')
model_output = model(**inputs)
mean_pooling(model_output, inputs['attention_mask'])

tensor([[ 3.6122e-02, -1.2337e-01, -7.5299e-03, -5.0620e+00,  1.6959e-01,
          9.2553e-02, -2.8838e-01,  2.1198e-01, -6.3860e-01, -3.6451e-01,
         -1.6556e-02,  1.8412e-01, -2.9757e-01,  5.1234e-02,  4.4390e-02,
          2.3413e-01,  3.8918e-01, -2.7811e-02,  2.5196e-01, -1.6740e-01,
         -5.0612e-02, -1.1393e-01, -5.8496e-02, -9.2304e-02, -3.3941e-01,
          2.9963e-02, -6.9979e-02, -1.2118e-01, -1.7573e-01,  2.4531e-01,
          1.6764e-01,  6.0146e-02, -1.3626e-01,  1.6598e-01,  1.2201e-01,
          5.2110e-02, -3.8716e-02, -3.4096e-02,  2.7250e-01, -6.6029e-02,
          2.2798e-01,  2.6542e-02,  3.8524e-01, -5.3627e-02,  9.2007e-02,
          1.2872e-01, -9.4732e-02,  1.4018e-01,  4.4361e-02,  3.0786e-02,
         -2.7000e-01,  2.5637e-02,  5.1933e-02,  1.9702e-01, -1.4610e-01,
         -2.9428e-01,  1.8972e-01, -1.1122e-01, -2.5106e-01,  1.7953e-01,
          1.1555e-01, -1.8627e-01,  1.9639e-02, -1.5012e-01,  6.5611e-02,
          6.9550e-02,  1.3693e-01,  3.

In [10]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import os
import json

In [15]:
def get_data(gallery):
  result = []
  for name in os.listdir(path='/content/drive/MyDrive/joinon/' + gallery):
    with open('/content/drive/MyDrive/joinon/' + gallery + '/' + name, 'r', encoding='utf-8') as f:
      data = json.load(f)
      valid = []
      for d in data['content'].split('\n'):
        d = d.replace('- dc official App', '').strip()
        if d and not d.replace(' ', '').isdecimal():
          valid.append(d)
      result.append('\n'.join([data['title']] + valid))
  return result

In [33]:
import random
cocktail = random.sample(get_data('cock_tail'),195) + random.sample(get_data('brandy'),195) + random.sample(get_data('ulisul'),195) + random.sample(get_data('wine'),195)
cocktail[:5]

['오늘의 입고~!\n컵크기가 예상보다 크네~\n오늘은 마이타이다~!',
 '운동하면 술 못마신대\n간이 씹창난다는거같음?\n근데 그냥 마실라고 ㅇㅇ...\n운동하고 시원한 맥주 참으면 스트레스받아서 암걸릴듯 ㅇㅈ??\n평생을 멸치랑 배불뚝 아조씨로 살다가 오늘부터 운동 시작햇는데 진짜 노무노무 힘들다 ㅠ',
 '드디어 샀다 노일리프랏\n달리러 샀는데 주변에 수령 가능한곳이 없어서 서울까지 와서 수령함...\n비싸고 귀한거니 아껴뒀다 담에 따야지 ㅋㅋㅋ',
 '칵테일 해먹고 나온 자몽씨앗\n"생매장"\n4개중에 1개만 올라와도 좋겠다',
 '인퓨징 해보려고 하는데 추천좀\n호불호 없게 기주랑 재료 기간 추천점']

In [34]:
class CustomTokenizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
  def __call__(self, target):
    return self.tokenizer.tokenize(target)

In [35]:
custom_tokenizer = CustomTokenizer(tokenizer)
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [36]:
# bertopic_model = BERTopic(embedding_model=model,
#                  vectorizer_model=vectorizer,
#                  nr_topics=50,
#                  top_n_words=10,
#                  calculate_probabilities=True)
bertopic_model = BERTopic(language='multilingual',
                          nr_topics=50,
                          top_n_words=10,
                          calculate_probabilities=True)

In [37]:
topics, probs = bertopic_model.fit_transform(cocktail)

In [38]:
bertopic_model.visualize_topics()

In [39]:
bertopic_model.visualize_distribution(probs[0])

In [40]:
for i in range(0, 50):
  print(i,'번째 토픽 :', bertopic_model.get_topic(i))

0 번째 토픽 : [('the', 0.0669593205534655), ('쌀전분', 0.04141376381523039), ('https', 0.038040539743446934), ('같다', 0.035042415535964176), ('of', 0.032834824620297014), ('starter', 0.03060948026185986), ('culture', 0.027760768175902957), ('ec', 0.027760768175902957), ('kr', 0.025860239450763278), ('and', 0.02429067215391509)]
1 번째 토픽 : [('이야', 0.03441690814645146), ('부랑디', 0.027686472372701623), ('추천좀요', 0.019922699274168513), ('추천', 0.01875801709218136), ('있는데', 0.018618028514919643), ('기합', 0.01823511521944584), ('가는', 0.01783272910393162), ('쓰고', 0.01783272910393162), ('없음', 0.01783272910393162), ('이거', 0.016782792049179384)]
2 번째 토픽 : [('순서', 0.4182948359398283), ('이미지', 0.4168152014014966), ('on', 0.4153556526657402), ('구형보틀', 0.1267282854217513), ('들은', 0.1196826691555328), ('받고', 0.1188498827075946), ('사제락', 0.11468545147390918), ('나눔릴', 0.08767627282178221), ('여자는', 0.06938853306424123), ('트위스트', 0.06938853306424123)]
3 번째 토픽 : [('ㅋㅋㅋ', 0.038608997536997224), ('가성비', 0.03213995427495

In [43]:
for i in range(8):
  print(topics.count(i))

13
88
13
17
167
75
49
55


In [44]:
print(topics)

[1, -1, 3, 1, 4, 4, 2, 4, -1, -1, -1, 4, -1, 4, -1, 1, 3, -1, 1, -1, 5, 2, 4, -1, 7, -1, -1, 5, 4, 1, 4, 4, 4, -1, 4, 4, 4, 1, 4, 4, -1, 4, 4, 4, -1, -1, -1, 1, 4, 4, 4, 4, 7, 1, 4, -1, 4, 7, -1, 5, 4, 4, 4, 4, 3, -1, 4, 1, 4, 1, 1, 5, 4, 4, 1, -1, 4, -1, 4, 1, 4, 1, 4, -1, 4, 3, 4, -1, 4, 4, -1, 4, -1, 4, 4, -1, 4, -1, 4, -1, 4, 4, 4, 7, 4, 7, -1, -1, 2, 4, 4, 4, -1, 4, -1, 4, 3, -1, 4, 4, 4, 4, 1, 4, 4, 4, -1, 4, 1, 4, 5, 4, 4, 4, -1, 4, 1, 4, 4, 2, -1, -1, 4, 1, 4, 4, -1, 4, 4, 5, 6, 5, 4, 4, 1, 4, 4, -1, -1, 4, 5, 4, -1, -1, 4, 5, 4, 1, 4, -1, 1, 1, 5, 4, 2, -1, 2, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, -1, -1, 5, 4, -1, 1, 4, -1, 5, 3, -1, 1, 7, 7, -1, -1, 7, -1, 4, 1, -1, -1, 1, 1, 5, -1, -1, 1, -1, 1, 4, 1, 3, 4, 4, -1, 3, -1, 5, 4, -1, 1, 1, 7, 6, -1, 7, 1, 7, 7, -1, 2, -1, -1, 7, -1, -1, 4, 1, 5, 3, -1, 2, 1, 2, -1, 1, -1, -1, -1, 1, -1, 7, 7, -1, 1, -1, -1, 4, 5, -1, 5, -1, 5, 4, 2, 4, -1, -1, 4, 4, -1, -1, 5, 5, 5, -1, -1, -1, -1, 1, 5, 4, 5, 4, 5, -1, -1, 0, -1, -1, 5, -1, 4, -1, 7,